In [1]:
import os
import sys
import re
import pandas as pd
from transformers import pipeline

from ngrams import *
from ldatops import *
from parse import *
from run_bert import *
from run_spacy import *

# Enter path to desired text file below

In [2]:
pth = "test2.txt"

# Peforming Pre-processing of book and displaying first 50 characters of text

In [ ]:
book, sents = preprocess(pth)
book[:50]

# Extracting unigrams, bigrams, and trigrams from text and displaying the first 5 terms and their associated confidence scores

In [ ]:
grams, gprobs = get_grams(sents)
print(grams[:5])
print(gprobs[:5])

# Extracting topics using Latent Dirichlet Association, and displaying the first 5 terms with associated confidence scores

In [ ]:
ldatops, ldaprobs = get_lda(sents)
print(ldatops[:5])
print(ldaprobs[:5])

# Extracting topics using BERTopic, and displaying the first 5 terms with associated confidence scores

In [ ]:
btops, bprob = get_bert(sents)
print(btops[:5])
print(bprob[:5])

# Extracting named entities using spaCy, and displaying the first 5 term categories

In [ ]:
ents = get_spacy(book)
print(ents[:5])

# Performing hierarchical summarization using HuggingFace's summarization pipeline, and displaying final summary

# !!!!! WARNING, THIS TAKES A LONG TIME TO PROCESS !!!!!

In [ ]:
segs = int(len(book) / 1024) + 1
summary = ""
summarizer = pipeline("summarization", model="google-t5/t5-large")

In [ ]:
iter = 0
while len(book.split(".")) > 15:
# for q in range(1):
    for i in range(segs):
        if i == 0:
            st = 0
            en = 1024
            outputs = summarizer(book[st:en], max_length=128, clean_up_tokenization_spaces=True)
            summary += outputs[0]['summary_text']
        if i == segs-1:
            st = i + (i*1023)
            remlen = len(book[st:])
            outputs = summarizer(book[st:], max_length=remlen, clean_up_tokenization_spaces=True)
            summary += " " + outputs[0]['summary_text']
        else:
            st = i + (i*1023)
            en = st + 1024
            outputs = summarizer(book[st:en], max_length=128, clean_up_tokenization_spaces=True)
            summary += " " + outputs[0]['summary_text']
    book = summary
    summary = ""
    print("\n***************************\n\n", iter, len(book))
    segs = int(len(book) / 1024) + 1
    iter += 1

print(book)